In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import warnings
import scipy
import os
from velocity.velocity_data import age_vel_disp, star_pos_vel
from simulation_data import get
from simulation_data.galaxies import GalaxyPopulation
my_galaxy_population = GalaxyPopulation()
from simulation_data.galaxies.galaxy import age_profile, get_star_formation_history, get_galaxy_particle_data, get_stellar_assembly_data

In [2]:
with h5py.File('galaxy_population_data_'+str(2)+'.hdf5', 'r') as f:
    ids = f['ids'][:]
    median_age = f['median_age'][:]
    halfmass_radius = f['halfmass_radius'][:]
    total_mass = f['total_mass'][:]
    newbin_current_SFR = f['newbin_current_SFR'][:]
    maximum_merger_ratio_30kpc_current_fraction = f['maximum_merger_ratio_30kpc_current_fraction'][:]
    lam = f['lam'][:]
    bulge_ratio = f['bulge_ratio'][:]
    ellipticity = f['ellipticity'][:]
    lambda_obs = f['lambda_obs'][:]
    eps_effec_test = f['ellip_eff'][:]
    vel_disp_young = f['vel_disp_young'][:]
    vel_disp_tot = f['vel_disp_tot'][:]
    vel_disp_old = f['vel_disp_old'][:]
    lambda_obs_eff = f['lam_obs_eff'][:]
    lambda_eff = f['lam_eff'][:]

In [3]:
warnings.filterwarnings('ignore')

PSB_selection = (median_age<=1.2)&(np.log10(newbin_current_SFR/10**(total_mass))<=-10.5)
intermediate_quiescent_selection = (median_age>1.2)&(median_age<=1.5)&(np.log10(newbin_current_SFR/10**(total_mass))<=-10.5)
old_quiescent_selection = (median_age>1.5)&(np.log10(newbin_current_SFR/10**(total_mass))<=-10.5)
starforming_selection = np.where(np.log10(newbin_current_SFR/10**(total_mass))>-10.5)[0]

In [4]:
vel_young = np.nan_to_num(vel_disp_young)
tot_selection = PSB_selection ^ old_quiescent_selection ^ intermediate_quiescent_selection

In [5]:
tot_young = vel_young[tot_selection]
tot_old = vel_disp_old[tot_selection]

deviate_ind = np.where(tot_young > tot_old)

print(len(tot_young[deviate_ind]))
print(ids[deviate_ind])

72
[    3     4  1503  1504  1505  1510  2973  2974  2977  4567  5586  5587
  6631  6632  7817  8712  8715  9918  9920  9924 10756 10757 10759 11770
 11771 12501 12504 13210 13211 13843 13847 14608 14610 16797 16798 18084
 18616 19044 19648 20651 21759 21761 22747 23223 23224 24399 25311 26122
 26571 27283 27285 27759 28114 28511 30097 30493 31278 33797 33798 34151
 34152 34632 34633 35253 35631 35956 36390 36578 36579 36839 36840 37177]


In [6]:
redshift = 2
scale_factor = 1.0 / (1+redshift)

In [7]:
yf = []

for i in ids[deviate_ind]:
    id = i
    url = "http://www.tng-project.org/api/TNG100-1/snapshots/z=" + str(redshift) + "/subhalos/" + str(id)
    sub = get(url)
    effec_r = sub['halfmassrad_stars']*scale_factor
    
    pos,vel,star_masses,cor_pos = star_pos_vel(id)
    radius = np.sqrt(pos[:,0]**2 + pos[:,1]**2 + pos[:,2]**2)
    
    stellar_data = get_galaxy_particle_data(id=id , redshift=2, populate_dict=True)
    LookbackTime = stellar_data['LookbackTime']

    young_ind = np.where((LookbackTime < 1) & (radius < effec_r))[0]
    old_ind = np.where((LookbackTime >= 1) & (radius < effec_r))[0]
    tot_ind = np.where((LookbackTime < 15) & (radius < effec_r))[0]

    young_num = len(young_ind)
    tot_num = len(tot_ind)

    young_frac = young_num/tot_num
    
    print(i)
    print(young_frac)
    
    yf.append(young_frac)

3
0.24027020327218643
4
0.4367841531566425
1503
7.361818663682676e-05
1504
0.5905489396024102
1505
0.40660919540229884
1510
6.047411707789066e-05
2973
0.3764353075800024
2974
0.42738610077441413
2977
0.36252328985893
4567
0.0
5586
0.08850505673609195
5587
0.4981359166299721
6631
0.3825827532319932
6632
0.3100425079702444
7817
0.0069249861541301715
8712
0.00026522853859075235
8715
0.43219625865751826
9918
0.27622994180920474
9920
0.00026438706265973383
9924
0.295314675890571
10756
1.4768649111665756e-05
10757
0.7113532186275161
10759
0.2172886985403374
11770
0.09875530343873795
11771
0.6361868526738533
12501
0.695373339441136
12504
0.20225958780175649
13210
0.0014819228929643946
13211
0.07519771813820822
13843
0.3865977299349679
13847
0.8739766533530463
14608
3.55495967660024e-05
14610
0.648127203778857
16797
0.018321154297459803
16798
0.04643244738578086
18084
0.0541138050288503
18616
0.7132281316968634
19044
0.022024331151090263
19648
0.5141165168265439
20651
0.020922303347568536
2175